In [2]:
# 模板槽值默认值
import sys
sys.path.append("..")
from static import annotate_db
annotate_db

{'acronym': ['ACL', 'EMNLP'],
 'year': ['今年', '2022', '去年', '近两年'],
 'fullName': ['XXX'],
 'paperDue': ['XXX',
  '最近一个月',
  '最近',
  '快要',
  '要',
  '下周',
  '下下个月',
  '下个月',
  '下一个月',
  '这几天',
  '11月份',
  '已经'],
 'noticeDue': ['XXX',
  '最近一个月',
  '最近',
  '快要',
  '要',
  '下周',
  '下下个月',
  '下个月',
  '下一个月',
  '这几天',
  '11月份',
  '已经'],
 'heldDate': ['XXX',
  '最近一个月',
  '最近',
  '快要',
  '要',
  '下周',
  '下下个月',
  '下个月',
  '下一个月',
  '这几天',
  '11月份',
  '已经'],
 'abstractDue': ['XXX', '11月份', '马上要', '最近要', '下个月', '已经'],
 'finalDue': ['XXX', '11月份', '马上要', '最近要', '最近', '下个月', '已经'],
 'location': ['XXX', '北京', '线上', '线下', '网上'],
 'category': ['XXX', 'NLP', '人工智能'],
 'ccf': ['XXX', 'A类', 'B类'],
 'website': ['XXX', 'https://www.2022.aclweb.org/', 'https://www.nature.com/'],
 'bestPaper': ['XXX'],
 'confRelated': ['XXX'],
 'journalRelated': ['XXX'],
 'doi': ['XXX'],
 'title': ['这篇论文', 'Attention is all you need', 'Generative Adversarial Nets'],
 'author': ['李航', '黄河燕', '毛先领'],
 'venue': ['ACL', 'EMNLP'],


In [4]:
# 检查annotate_db是否全了
import json
with open("../annotate/user_template.json", "r", encoding="utf-8") as fw:
    user=json.load(fw)
with open("../annotate/system_template.json", "r", encoding="utf-8") as fw:
    system=json.load(fw)

for _, i in user.items():
    required_slot = i["required_slot"]
    for slot in required_slot:
        if slot=="OA":
            continue
        if "Doubt" in i["action"][0]:
            continue
        if i["action"][0]=="Paper-Download" and slot in ["title", "pdf"]:
            continue
        for k in i["message"]:
            value = None
            for j in annotate_db[slot]:
                if j in k:
                    value = j
                    break
            if value is None:
                print(k, annotate_db[slot],slot,i["action"])

In [8]:
# 生成data.json
import json
time_candidate=[]
for year in range(2010, 2024): # XX年
    t=[(year,1,1), (year,12,31), "%d年" % year]
    time_candidate.append(t)
for year in [2022, 2023]: # 一个月时间段
    for month in range(1,13):
        # 一个月
        t=[(year,month,1), (year,month,31), "%d年%d月" % (year,month)]
        time_candidate.append(t)
        # # 两个月
        # if month<12:
        #     t=[(year,month,1), (year,month+1,31), "从%d年%d月到%d月" % (year,month,month+1), "%d年%d月和%d月" % (year,month,month+1)]
        # else:
        #     t=[(year,month,1), (year+1,month,31), "从%d年%d月到%d年%d月" % (year,month,year+1,1), "%d年%d月和%d年%d月" % (year,month,year,1)]
        # time_candidate.append(t)
for year in [2021, 2022, 2023]:
    # 三个月            
    if month<11:
        t=[(year,month,1), (year,month+2,31), "从%d年%d月到%d月" % (year,month,month+2)]
    else:
        t=[(year,month,1), (year+1,(month+2)%12,31), "从%d年%d月到%d年%d月" % (year,month,year+1,(month+2)%12)]
    time_candidate.append(t)

months = [i for i in range(11,13)]
days = [0,31,28,31,30,31,30,31,31,30,31,30,31]
time = [(x,i,y,j) for x in months for y in months for i in range(1,days[x]+1) for j in range(1,days[x]+1)]
for t in time: # 2022年XX月XX日到XX月XX日
    if t[0]<t[2]:
        x=[(2022,t[0],t[1]), (2022,t[2],t[3]), "从%d月%d号到%d月%d号" % (t[0],t[1],t[2],t[3])]
        time_candidate.append(x)
    elif t[0]==12 and t[0]==t[2] and t[1]<t[3]:
        x=[(2022,t[0],t[1]), (2022,t[2],t[3]), "从%d月%d号到%d号" % (t[0],t[1],t[3])]
        time_candidate.append(x)

print("时间模板数:", len(time_candidate))
with open("date.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
result=[]
# 结合之前的人工标注填入值
for time in time_candidate:
    value = time
    for i in data:
        if time[0][0]==i[0][0] and time[0][1]==i[0][1] and time[0][2]==i[0][2] and time[1][0]==i[1][0] and time[1][1]==i[1][1] and time[1][2]==i[1][2]:
            value1, value2 = time[2:], i[2:]
            value += list(set(value2) - set(value1))
            break
    result.append(value)
    
with open("z.json", "w", encoding="utf-8") as fw:
    json.dump(result,fw,indent=4,ensure_ascii=False)

时间模板数: 1406


In [6]:
# 合并date.json中相同时间段
import json
with open("date.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
days = [0,31,28,31,30,31,30,31,31,30,31,30,31]
result = {}  # 相同slot合并
for i in data:
    start,end = i[0],i[1]
    start[2]=days[start[1]] if start[2]>days[start[1]] else start[2]
    end[2]=days[end[1]] if end[2]>days[end[1]] else end[2]
    key = "%d-%d-%d %d-%d-%d" % (start[0],start[1],start[2],end[0],end[1],end[2])
    if key in result.keys():
        result[key].extend(i[2:])
    else:
        result[key] = i
d = [j for _,j in result.items()]
with open("date.json", "w", encoding="utf-8") as fw:
    json.dump(d,fw,indent=4,ensure_ascii=False)

In [27]:
# 生成location.json
import jieba
import re
import json
with open("../dataset/db_process/conference_db.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
location={}
for i in data:
    pattern = re.compile(r'[a-zA-Z]+|[,，&的\s]|-|（.*）')
    i["location"]=re.sub(pattern, "", i["location"])
    seg_list=jieba.cut(i["location"], cut_all=False)
    for j in seg_list:
        country=j
        break
    if i["location"] == "":
        continue
    elif i["location"] not in location.keys():
        location[i["location"]] = 1
    else:
        location[i["location"]] += 1
    if country == "":
        continue
    elif country not in location.keys():
        location[country] = 1
    else:
        location[country] += 1
location = sorted(location.items(),key=lambda x:x[1],reverse=True)
location = [i[0] for i in location[:29]]
with open("location.json", "w", encoding="utf-8") as fw:
    json.dump(location,fw,indent=4,ensure_ascii=False)

In [11]:
# 生成category.json
import re
with open("../dataset/db_process/conference_db.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
category = {}
pattern = re.compile(r'[0-9一的（），,司或坐]')
for i in data:
    for j in i["category"]:
        if re.search(pattern,j) is not None:
            continue
        if j == "":
            continue
        if j not in category.keys():
            category[j] = 1
        else:
            category[j] += 1
category = sorted(category.items(),key=lambda x:x[1],reverse=True)
category = [i[0] for i in category[:29]]
with open("conf_category.json", "w", encoding="utf-8") as fw:
    json.dump(category,fw,indent=4,ensure_ascii=False)

In [31]:
# 生成venue.json、author.json、paper_institution.json
with open("../dataset/db_process/paper_db.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
author = {}
institution = {}
venue = {}
for i in data:
    if i["venue"] != "":
        if i["venue"] not in venue.keys():
            venue[i["venue"]] = 1
        else:
            venue[i["venue"]] += 1
    for j in i["author"]:
        if j == "":
            continue
        if j not in author.keys():
            author[j] = 1
        else:
            author[j] += 1
    for j in i["institution"]:
        if j == "":
            continue
        if j not in institution.keys():
            institution[j] = 1
        else:
            institution[j] += 1
author = sorted(author.items(),key=lambda x:x[1],reverse=True)
author = [i[0] for i in author if i[1]>=100]
venue = sorted(venue.items(),key=lambda x:x[1],reverse=True)
venue = [i[0] for i in venue if i[1]>=100]
institution = sorted(institution.items(),key=lambda x:x[1],reverse=True)
institution = [i[0] for i in institution if i[1]>=100]

with open("venue.json", "w", encoding="utf-8") as fw:
    json.dump(venue,fw,indent=4,ensure_ascii=False)
with open("author.json", "w", encoding="utf-8") as fw:
    json.dump(author,fw,indent=4,ensure_ascii=False)
with open("paper_institution.json", "w", encoding="utf-8") as fw:
    json.dump(institution,fw,indent=4,ensure_ascii=False)

In [29]:
# 生成journal_category.json
with open("../dataset/db_process/journal_db.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
category = {}
for i in data:
    if i["category"] == "":
        continue
    if i["category"] not in category.keys():
        category[i["category"]] = 1
    else:
        category[i["category"]] += 1

with open("journal_category.json", "w", encoding="utf-8") as fw:
    json.dump(list(category.keys()),fw,indent=4,ensure_ascii=False)

In [40]:
# 生成field.json、affiliation.json、position.json
with open("../dataset/db_process/author_db.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
field = {}
affiliation = {}
position = []
for i in data:
    if i["position_zh"] != "" and i["position_zh"] not in position:
        position.append(i["position_zh"])
    for j in i["field"]:
        if j == "":
            continue
        if j not in field.keys():
            field[j] = 1
        else:
            field[j] += 1
    for j in i["affiliation"]:
        if j == "":
            continue
        if j not in affiliation.keys():
            affiliation[j] = 1
        else:
            affiliation[j] += 1
field = sorted(field.items(),key=lambda x:x[1],reverse=True)
field = [i[0] for i in field[:30]]
affiliation = sorted(affiliation.items(),key=lambda x:x[1],reverse=True)
affiliation = [i[0] for i in affiliation if i[1]>1]

with open("field.json", "w", encoding="utf-8") as fw:
    json.dump(field,fw,indent=4,ensure_ascii=False)
with open("affiliation.json", "w", encoding="utf-8") as fw:
    json.dump(affiliation,fw,indent=4,ensure_ascii=False)
with open("position.json", "w", encoding="utf-8") as fw:
    json.dump(position,fw,indent=4,ensure_ascii=False)

In [41]:
# ccf
ccf = ["A类", "B类", "C类", "B类以上", "B类以下",  "C类以上", "A类以下", "ccf-a", "ccf-b", "ccf-c", "ccf a", "ccf b", "ccf c", "ccfA", "ccfB", "ccfC"]
sci = ["sci 1区", "sci 2区", "sci 3区", "sci 4区", "sci1区", "sci2区", "sci3区", "sci4区", "1区", "2区", "3区", "4区"]
year = ["今年", "去年", "前年", "2022", "2021", "2020", "2019", "2018", "2017", "2016", "2015", "2014", "2013", "2012", "2011", "2010", "2009", "2008", "2007", "2006", "2005", "2004", "2003", "2002", "2001", "2022年", "2021年", "2020年", "2019年", "2018年", "2017年", "2016年", "2015年", "2014年", "2013年", "2012年", "2011年", "2010年", "2009年", "2008年", "2007年", "2006年", "2005年", "2004年", "2003年", "2002年", "2001年"]
OA = ["可以开放访问", "不可以开放访问", "禁止访问"]
# if h_index
IF=['IF大于1', 'IF>1', 'IF影响因子大于1', 'IF影响因子>1', 'IF大于2', 'IF>2', 'IF影响因子大于2', 'IF影响因子>2', 'IF大于3', 'IF>3', 'IF影响因子大于3', 'IF影响因子>3', 'IF大于4', 'IF>4', 'IF影响因子大于4', 'IF影响因子>4', 'IF大于5', 'IF>5', 'IF影响因子大于5', 'IF影响因子>5', 'IF大于6', 'IF>6', 'IF影响因子大于6', 'IF影响因子>6', 'IF大于7', 'IF>7', 'IF影响因子大于7', 'IF影响因子>7', 'IF大于8', 'IF>8', 'IF影响因子大于8', 'IF影响因子>8', 'IF大于9', 'IF>9', 'IF影响因子大于9', 'IF影响因子>9', 'IF大于10', 'IF>10', 'IF影响因子大于10', 'IF影响因子>10']
h_index=['h指数大于10', 'h指数>10', 'h5指数大于10', 'h5指数>10', 'h指数大于20', 'h指数>20', 'h5指数大于20', 'h5指数>20', 'h指数大于30', 'h指数>30', 'h5指数大于30', 'h5指数>30', 'h指数大于40', 'h指数>40', 'h5指数大于40', 'h5指数>40', 'h指数大于50', 'h指数>50', 'h5指数大于50', 'h5指数>50', 'h指数大于60', 'h指数>60', 'h5指数大于60', 'h5指数>60', 'h指数大于70', 'h指数>70', 'h5指数大于70', 'h5指数>70', 'h指数大于80', 'h指数>80', 'h5指数大于80', 'h5指数>80', 'h指数大于90', 'h指数>90', 'h5指数大于90', 'h5指数>90', 'h指数大于100', 'h指数>100', 'h5指数大于100', 'h5指数>100']
# IF, h_index=[],[]
# for i in range(1,11):
#     num = str(i)
#     num1 = str(i*10)
#     IF += ["IF大于"+num,"IF>"+num,"IF影响因子大于"+num,"IF影响因子>"+num]
#     h_index += ["h指数大于"+num1,"h指数>"+num1,"h5指数大于"+num1,"h5指数>"+num1]
# print(IF)
# print(h_index)


In [42]:
# 将除date外合并到constraints.json中
import json
dict = {
    "Conference": {},
    "Paper": {},
    "Journal": {},
    "Author": {}
}
# with open("constraints/date.json", "r", encoding="utf-8") as fw:
#     dict["Conference"]["date"] = json.load(fw)
with open("constraints/location.json", "r", encoding="utf-8") as fw:
    dict["Conference"]["location"] = json.load(fw)
with open("constraints/conf_category.json", "r", encoding="utf-8") as fw:
    dict["Conference"]["category"] = json.load(fw)
dict["Conference"]["ccf"]=ccf
with open("constraints/author.json", "r", encoding="utf-8") as fw:
    dict["Paper"]["author"] = json.load(fw)
with open("constraints/paper_institution.json", "r", encoding="utf-8") as fw:
    dict["Paper"]["institution"] = json.load(fw)
with open("constraints/venue.json", "r", encoding="utf-8") as fw:
    dict["Paper"]["venue"] = json.load(fw)
dict["Paper"]["year"] = year
dict["Journal"]["IF"] = IF
dict["Journal"]["h_index"] = h_index
dict["Journal"]["ccf"] = ccf
dict["Journal"]["sci"] = sci
dict["Journal"]["OA"] = OA
with open("constraints/journal_category.json", "r", encoding="utf-8") as fw:
    dict["Journal"]["category"] = json.load(fw)
with open("constraints/field.json", "r", encoding="utf-8") as fw:
    dict["Author"]["field"] = json.load(fw)
with open("constraints/affiliation.json", "r", encoding="utf-8") as fw:
    dict["Author"]["affiliation"] = json.load(fw)
with open("constraints/position.json", "r", encoding="utf-8") as fw:
    dict["Author"]["position"] = json.load(fw)
# 写入
with open("constraints.json", "w", encoding="utf-8") as fw:
    json.dump(dict,fw,indent=4,ensure_ascii=False)
